In [1]:
import pandas as pd
import numpy as np


df_despesas = pd.read_csv('consolidado_despesas.csv', sep=';', encoding='utf-8')
df_cadastro = pd.read_csv('Relatorio_cadop.csv', sep=';', encoding='latin-1')

def limpar_cnpj(cnpj):
    return str(cnpj).replace('.', '').replace('/', '').replace('-', '').strip()

df_despesas['CNPJ_clean'] = df_despesas['CNPJ'].apply(limpar_cnpj)
df_cadastro['CNPJ_clean'] = df_cadastro['CNPJ'].apply(limpar_cnpj)

df_cadastro_unique = df_cadastro.drop_duplicates(subset='CNPJ_clean', keep='first')

df_final = pd.merge(
    df_despesas,
    df_cadastro_unique[['CNPJ_clean', 'REGISTRO_OPERADORA', 'Modalidade', 'UF']],
    on='CNPJ_clean',
    how='left'
)

df_final = df_final.rename(columns={'REGISTRO_OPERADORA': 'RegistroANS'})

df_final['UF'] = df_final['UF'].fillna('N/A')

agregado = df_final.groupby(['RazaoSocial', 'UF']).agg(
    Total_Despesas=('ValorDespesas', 'sum'),
    Media_Trimestral=('ValorDespesas', 'mean'),
    Desvio_Padrao_Despesas=('ValorDespesas', 'std')
).reset_index()

agregado = agregado.sort_values(by='Total_Despesas', ascending=False)

agregado.to_csv('despesas_agregadas.csv', index=False, sep=';', encoding='utf-8-sig')

print("Processamento concluído! Arquivo 'despesas_agregadas.csv' gerado.")
agregado.head()

Processamento concluído! Arquivo 'despesas_agregadas.csv' gerado.


RazaoSocial   UF  Total_Despesas  Media_Trimestral  \
980    RAZAO_5711  N/A    4.359374e+11      1.073737e+09   
982    RAZAO_6246  N/A    3.395880e+11      5.208405e+08   
189  RAZAO_326305  N/A    2.323330e+11      3.878680e+08   
440  RAZAO_359017  N/A    1.019908e+11      2.377407e+08   
492  RAZAO_368253  N/A    9.546143e+10      2.267492e+08   

     Desvio_Padrao_Despesas  
980            3.514179e+09  
982            1.870911e+09  
189            1.830582e+09  
440            1.002788e+09  
492            8.900836e+08